# Schedule Vaccination of each employees of the company. Where vaccination is done in the descending order of employees count per city. Vaccination of 200 employees is possible in one day per city. Vaccine can be done at atmost one city on a single day

In [ ]:
import os


In [2]:
import sys
!{sys.executable} -m pip install findspark

In [3]:
os.environ["SPARK_HOME"]=r"C:\Users\pawan palariya\Downloads\spark-3.1.2-bin-hadoop3.2\spark-3.1.2-bin-hadoop3.2"

In [4]:
os.environ["JAVA_HOME"] =r"C:\Program Files\Java\jdk1.8.0_291"
os.environ["HADOOP_HOME"] =r"C:\Hadoop"

In [5]:
import findspark
findspark.init()

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.appName('interview_code').getOrCreate()

In [61]:
df_input=spark.read.format("csv").option("header","true").load(r"C:\Users\pawan palariya\Downloads\50000-Records")


In [73]:
df=df_input.withColumn("Full_name", concat_ws(" ",col("First_Name"),col("Middle_Initial"),col("Last_Name")))

In [74]:
df.count()


50000

In [75]:
df.select("Emp_Id","Full_Name").show(4)

+------+-------------------+
|Emp_Id|          Full_Name|
+------+-------------------+
|158108|Bernard N Weatherly|
|863655|     Cordia M Knopp|
|343966|       Burton C Jin|
|389958|     Lauren O Guzzi|
+------+-------------------+
only showing top 4 rows



In [76]:
df.printSchema()

root
 |-- Emp_ID: string (nullable = true)
 |-- Name_Prefix: string (nullable = true)
 |-- First_Name: string (nullable = true)
 |-- Middle_Initial: string (nullable = true)
 |-- Last_Name: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- E Mail: string (nullable = true)
 |-- Father's Name: string (nullable = true)
 |-- Mother's Name: string (nullable = true)
 |-- Mother's Maiden Name: string (nullable = true)
 |-- Date of Birth: string (nullable = true)
 |-- Time of Birth: string (nullable = true)
 |-- Age in Yrs.: string (nullable = true)
 |-- Weight in Kgs.: string (nullable = true)
 |-- Date of Joining: string (nullable = true)
 |-- Quarter of Joining: string (nullable = true)
 |-- Half of Joining: string (nullable = true)
 |-- Year of Joining: string (nullable = true)
 |-- Month of Joining: string (nullable = true)
 |-- Month Name of Joining: string (nullable = true)
 |-- Short Month: string (nullable = true)
 |-- Day of Joining: string (nullable = true)
 |-- DO

In [77]:
df_citywise_employees=df.groupby("city").count().sort(col("count").desc())

In [78]:
df_citywise_employees.show()

+-------------+-----+
|         city|count|
+-------------+-----+
|   Washington|  368|
|      Houston|  206|
|New York City|  196|
|      El Paso|  188|
|       Dallas|  160|
|  Springfield|  147|
|      Atlanta|  143|
|        Miami|  137|
|   Sacramento|  134|
|     Richmond|  124|
|  San Antonio|  115|
|  Los Angeles|  113|
|       Austin|  108|
|       Denver|  107|
|  Kansas City|  104|
| Philadelphia|   99|
|    Las Vegas|   99|
|      Phoenix|   98|
|      Chicago|   98|
|       Albany|   96|
+-------------+-----+
only showing top 20 rows



In [79]:
list_rows=df_citywise_employees.take(df_citywise_employees.count())

In [80]:
type(list_rows)

list

In [82]:
len(list_rows)

14552

In [140]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,DateType
from datetime import date,timedelta
from pyspark.sql import Row
date_today=date.today()+timedelta(days=1)
id=0
rows_list=[]
vaccination_schema = StructType([ \
    StructField("empid",StringType(),True), \
    StructField("empname",StringType(),True), \
    StructField("vaccination_id",StringType(),True), \
    StructField("vaccination_date", DateType(), True), \
    StructField("city", StringType(), True),\
    StructField("vaccination_time", StringType(), True)])
for row in list_rows:
    city=row["city"]
    df_filter_by_city=df.filter(col("city")==city).select("Emp_ID","Full_name")
    city_total_emp=row["count"]
    list_emp_details=df_filter_by_city.take(city_total_emp)
    for i in range(city_total_emp):
        day=i//200
        date_today=date_today+timedelta(days=day)
        id=id+1
        vaccination_id="emp_00"+str(id)
        empid=list_emp_details[i]["Emp_ID"]
        empname=list_emp_details[i]["Full_name"]
        if(i%4==0):
            vaccination_time="9am-11am"
        elif(i%4==1):
            vaccination_time="11am-1pm"
        elif(i%4==2):
            vaccination_time="2pm-4pm"
        else:
            vaccination_time="4pm-6pm"
        row=Row(empid=empid,empname=empname,vaccination_id=vaccination_id,vaccination_date=date_today,city=city,vaccination_time=vaccination_time)
        rows_list.append(row)
df=spark.createDataFrame(rows_list,vaccination_schema)


In [142]:
df.count()

50000

In [145]:
df.show(1)

+------+--------------+--------------+----------------+----------+----------------+
| empid|       empname|vaccination_id|vaccination_date|      city|vaccination_time|
+------+--------------+--------------+----------------+----------+----------------+
|188762|Ellis V Mcneel|       emp_001|      2022-06-03|Washington|        9am-11am|
+------+--------------+--------------+----------------+----------+----------------+
only showing top 1 row

